In [1]:
import pandas as pd
import numpy as np
import random
from numpy.random import RandomState
from scipy import stats
from scipy.optimize import minimize
import scipy
import os
os.environ['R_HOME'] = '/Library/Frameworks/R.framework/Versions/4.1/Resources/'
import rpy2
from rpy2.robjects.packages import importr
from rpy2.robjects.packages import SignatureTranslatedAnonymousPackage
import datetime
from rpy2.robjects.vectors import FloatVector
import multiprocessing

np.random.seed(881)

# Define the model that generates pair simulations.
yuima = importr("yuima")
n_ou_sim_string = """
n_sim_ou = function(random_seed, num_sim,
                    mu11, mu12, mu21, mu22, sigma11, sigma12, sigma21, sigma22,
                    xinit_vec, T0, T, length){

  set.seed(random_seed)

  drift = c("mu11-mu12*X1", "mu21-mu22*X2")
  diffusion = matrix(c("exp(sigma11)", "exp(sigma12)", "exp(sigma21)", "exp(sigma22)"), 2, 2, byrow=TRUE)
  ou_model = setModel(drift=drift, diffusion=diffusion, 
                        time.variable = "t",
                        state.var=c("X1","X2"), solve.variable=c("X1","X2"))

  newsamp = setSampling(Initial=T0, Terminal=T, n=length)

  n_sim_ou_data = data.frame(matrix(nrow=length+1, ncol=2*num_sim))
  for (i in 1:num_sim){
    ou_sim = simulate(ou_model, 
                      true.par=list(
                        mu11=mu11, mu12=mu12, mu21=mu21, mu22=mu22, 
                        sigma11=sigma11, sigma12=sigma12, sigma21=sigma21, sigma22=sigma22), 
                      xinit=xinit_vec[i], sampling=newsamp)
    original_data = ou_sim@data@original.data
    one_sim_ou = data.frame(original_data[,1], original_data[,2])
    colnames(one_sim_ou) = c('series1', 'series2')
    n_sim_ou_data[, (2*i-1):(2*i)] = one_sim_ou
  }
  return(n_sim_ou_data)
}
"""

n_ou_sim = SignatureTranslatedAnonymousPackage(n_ou_sim_string, "n_ou_sim")
def n_ou_simulation(random_seed, num_sim,
                    mu11, mu12, mu21, mu22, sigma11, sigma12, sigma21, sigma22,
                    xinit_vec, T0, T, length):
    """num_sim simulations of bivariate Ornstein-Uhlenbeck process,
    length = length of one series
    """
    n_ou_sim_data = pd.DataFrame(
        n_ou_sim.n_sim_ou(random_seed=random_seed, num_sim=num_sim,
                              mu11=mu11, mu12=mu12, mu21=mu21, mu22=mu22,
                              sigma11=sigma11, sigma12=sigma12, sigma21=sigma21, sigma22=sigma22,
                              xinit_vec=xinit_vec, T0=T0, T=T, length=length)).transpose()
    return n_ou_sim_data

def price_to_log_price(n_price):
    return(np.log(n_price))

def log_price_to_price(n_log_price):
    return(np.exp(n_log_price))

def price_to_return(n_price):
    n_return = pd.DataFrame()
    for i in range(n_price.shape[1]):
        ith_column_price_series = n_price.iloc[:, i]
        n_return = pd.concat([n_return, 100 * (np.log(ith_column_price_series[1:].values) - np.log(ith_column_price_series[:-1]))], axis=1)
    return n_return

def log_price_to_return(n_log_price):
    n_real_return = pd.DataFrame()
    for i in range(n_log_price.shape[1]):
        ith_column_price_series = n_log_price.iloc[:, i]
        n_real_return = pd.concat([n_real_return, 100 * (ith_column_price_series[1:].values - ith_column_price_series[:-1])], axis=1)
    return n_real_return

# Define the function of transforming returns data into feature statistics (or moments).
def cal_stats(n_return, n_price=None):
    """ 'mean' and 'sd' checked
    'skewness' and 'kurtosis' checked
    (different expressions of calculation from intro to stat finance)
    8 statistics
    """
    return_series1 = n_return.iloc[:, ::2]
    return_series2 = n_return.iloc[:, 1::2]

    mean1 = return_series1.mean(axis=0).values
    sd1 = return_series1.std(axis=0).values
    skew1 = return_series1.skew(axis=0).values
    kurtosis1 = return_series1.kurtosis(axis=0).values

    mean2 = return_series2.mean(axis=0).values
    sd2 = return_series2.std(axis=0).values
    skew2 = return_series2.skew(axis=0).values
    kurtosis2 = return_series2.kurtosis(axis=0).values

    stats_data = pd.DataFrame([mean1, mean2, sd1, sd2, skew1, skew2, kurtosis1, kurtosis2])
    stats_data = stats_data.transpose()
    stats_data.columns = [
        'return_mean1', 'return_mean2',
        'return_sd1', 'return_sd2',
        'return_skew1', 'return_skew2',
        'return_kurtosis1', 'return_kurtosis2']
    return stats_data

def loss_function(params):
    """n_real_stats is a global amount calculated outside the function"""
    params = FloatVector(params)
    print(params)
    moment_loss = pd.DataFrame().reindex_like(real_stats)


    n_sim_log_price = n_ou_simulation(
        random_seed=int(np.random.randint(low=0, high=980608, size=(1,))), num_sim=num_sim,
        mu11=params[0], mu12=params[1], mu21=params[2], mu22=params[3],
        sigma11=params[4], sigma12=params[5], sigma21=params[6], sigma22=params[7],
        xinit_vec=xinit_vec, T0=T0, T=T, length=length)
    n_sim_price = log_price_to_price(n_sim_log_price)
    n_sim_return = price_to_return(n_sim_price)
    n_sim_stats = cal_stats(n_sim_return)


    for i in range(n_real_stats.shape[0]):
        for j in range(n_real_stats.shape[1]):
            moment_loss.iloc[i, j] = np.sqrt((n_real_stats.iloc[i, j] - n_sim_stats.iloc[i, j])**2)
    sum_all = np.sum(moment_loss)
    print(sum_all)
    print(np.sum(sum_all))
    print('---')

    return np.sum(sum_all)



real_price = pd.read_csv("sp500_20180101_20181231_pair_prices.csv", index_col=[0])
real_log_price = price_to_log_price(n_price=real_price)
real_return = pd.read_csv("sp500_20180101_20181231_pair_returns.csv", index_col=[0])
real_stats = cal_stats(n_return=real_return, n_price=None)


xinit_vec = []
for i in range(int(real_log_price.shape[1]/2)):
    init_pair_log_price = [real_log_price.iloc[0, 2*i], real_log_price.iloc[0, 2*i+1]]
    init_pair_log_price = FloatVector(init_pair_log_price)
    xinit_vec.append(init_pair_log_price)
num_sim, T0, T, length = real_stats.shape[0], 0, 1, real_price.shape[0]

n_real_stats = real_stats



initial0 = [1, 1, 1, 1, -1, -1, -1, -1]
begin_time = datetime.datetime.now()
res = minimize(loss_function, initial0, method='Powell',
               tol=1e-6, options={'disp': True},
               bounds=[(0., None), (0., None), (0., None), (0., None),
                       (None, None), (None, None), (None, None), (None, None)])
print(res.x)

time = datetime.datetime.now() - begin_time
print(time)

params = (res.x)
loss = loss_function((params))
print(loss)

[1]  1  1  1  1 -1 -1 -1 -1

return_mean1         201.809004
return_mean2         190.217593
return_sd1           391.363406
return_sd2           380.505649
return_skew1         161.925681
return_skew2         219.085093
return_kurtosis1    1117.956703
return_kurtosis2    1709.255110
dtype: float64
4372.118239554286
---
[1]  1.115092  1.000000  1.000000  1.000000 -1.000000 -1.000000 -1.000000
[8] -1.000000

return_mean1         199.897227
return_mean2         195.531244
return_sd1           388.362388
return_sd2           376.358229
return_skew1         163.112152
return_skew2         220.228290
return_kurtosis1    1124.637581
return_kurtosis2    1716.067123
dtype: float64
4384.194233737726
---
[1]  1.793574  1.000000  1.000000  1.000000 -1.000000 -1.000000 -1.000000
[8] -1.000000

return_mean1         156.549972
return_mean2         194.307825
return_sd1           388.260817
return_sd2           376.190760
return_skew1         161.241635
return_skew2         220.047106
return_kurtosis

return_mean1          60.554113
return_mean2         190.957807
return_sd1           395.713518
return_sd2           386.456388
return_skew1         157.832815
return_skew2         216.138644
return_kurtosis1    1129.755349
return_kurtosis2    1721.237014
dtype: float64
4258.645648929205
---
[1]  3.792831  1.000000  1.000000  1.000000 -1.000000 -1.000000 -1.000000
[8] -1.000000

return_mean1          53.041868
return_mean2         193.410359
return_sd1           387.398105
return_sd2           377.702696
return_skew1         161.141820
return_skew2         215.375319
return_kurtosis1    1123.309567
return_kurtosis2    1714.109673
dtype: float64
4225.489407405526
---
[1]  3.792655  1.000000  1.000000  1.000000 -1.000000 -1.000000 -1.000000
[8] -1.000000

return_mean1          59.017010
return_mean2         194.192020
return_sd1           390.043133
return_sd2           380.129061
return_skew1         160.206585
return_skew2         219.195635
return_kurtosis1    1122.226934
return_kurto

return_mean1          51.934993
return_mean2         191.458310
return_sd1           386.283207
return_sd2           377.350465
return_skew1         156.234530
return_skew2         219.250926
return_kurtosis1    1112.785295
return_kurtosis2    1705.785224
dtype: float64
4201.082949350495
---
[1]  3.7927220  0.9364733  1.0000000  1.0000000 -1.0000000 -1.0000000 -1.0000000
[8] -1.0000000

return_mean1          52.580442
return_mean2         191.558376
return_sd1           390.724765
return_sd2           381.550570
return_skew1         161.662587
return_skew2         217.325512
return_kurtosis1    1121.830615
return_kurtosis2    1712.698149
dtype: float64
4229.93101651259
---
[1]  3.7927220  0.9366174  1.0000000  1.0000000 -1.0000000 -1.0000000 -1.0000000
[8] -1.0000000

return_mean1          49.820854
return_mean2         193.838574
return_sd1           391.023102
return_sd2           380.057033
return_skew1         165.322389
return_skew2         224.494459
return_kurtosis1    1129.9648

return_mean1          52.305390
return_mean2          51.307544
return_sd1           391.584688
return_sd2           378.941387
return_skew1         158.582350
return_skew2         216.532498
return_kurtosis1    1129.967923
return_kurtosis2    1720.815177
dtype: float64
4100.036956721737
---
[1]  3.7927220  0.9366171  4.1250153  1.0000000 -1.0000000 -1.0000000 -1.0000000
[8] -1.0000000

return_mean1          49.908348
return_mean2          48.508974
return_sd1           386.808112
return_sd2           375.524166
return_skew1         158.251897
return_skew2         214.623134
return_kurtosis1    1123.560288
return_kurtosis2    1714.697478
dtype: float64
4071.8823969108653
---
[1]  3.7927220  0.9366171  4.1254566  1.0000000 -1.0000000 -1.0000000 -1.0000000
[8] -1.0000000

return_mean1          50.187250
return_mean2          50.371720
return_sd1           389.635452
return_sd2           377.442741
return_skew1         159.640154
return_skew2         216.708706
return_kurtosis1    1120.60

return_mean1          52.480313
return_mean2          56.945422
return_sd1           391.651195
return_sd2           380.055115
return_skew1         159.982239
return_skew2         214.970480
return_kurtosis1    1121.507275
return_kurtosis2    1711.689881
dtype: float64
4089.2819216909056
---
[1]  3.7927220  0.9366171  4.1250191  1.0902738 -1.0000000 -1.0000000 -1.0000000
[8] -1.0000000

return_mean1          48.515332
return_mean2          53.907987
return_sd1           390.174384
return_sd2           379.601115
return_skew1         159.338685
return_skew2         216.495923
return_kurtosis1    1126.623785
return_kurtosis2    1717.797613
dtype: float64
4092.454822892582
---
[1]  3.7927220  0.9366171  4.1250191  1.0901412 -1.0000000 -1.0000000 -1.0000000
[8] -1.0000000

return_mean1          54.272640
return_mean2          54.432523
return_sd1           393.635540
return_sd2           381.656754
return_skew1         160.379979
return_skew2         218.458155
return_kurtosis1    1124.27

...

return_mean1          47.373100
return_mean2          64.491243
return_sd1            87.012866
return_sd2            97.433628
return_skew1         160.567856
return_skew2         221.795409
return_kurtosis1    1124.097065
return_kurtosis2    1714.574911
dtype: float64
3517.346078972283
---
[1]  4.1337739  0.9921593  4.4425780  1.9578960 -2.4726376 -1.4196012 -3.5824183
[8] -1.3556774

return_mean1          44.418903
return_mean2         158.265983
return_sd1            83.538723
return_sd2            94.499662
return_skew1         158.237037
return_skew2         215.859702
return_kurtosis1    1120.353666
return_kurtosis2    1718.914094
dtype: float64
3594.087771459117
---
[1]  4.1337739  0.9921593  4.4425780  0.9090937 -2.4726376 -1.4196012 -3.5824183
[8] -1.3556774

return_mean1          44.773519
return_mean2          59.764410
return_sd1            85.377784
return_sd2            95.223245
return_skew1         156.496040
return_skew2         216.603580
return_kurtosis1    1119.703

return_mean1          48.900491
return_mean2          62.059729
return_sd1           131.245006
return_sd2            95.776884
return_skew1         158.688301
return_skew2         215.067667
return_kurtosis1    1128.444523
return_kurtosis2    1719.854181
dtype: float64
3560.036782327224
---
[1]  4.1337739  0.9921593  4.4425780  0.9090925 -4.0906716 -1.4196012 -3.5824183
[8] -1.3556774

return_mean1          44.777280
return_mean2          59.814938
return_sd1            88.123148
return_sd2            96.050024
return_skew1         159.701179
return_skew2         214.409688
return_kurtosis1    1120.297422
return_kurtosis2    1711.592305
dtype: float64
3494.765984189787
---
[1]  4.1337739  0.9921593  4.4425780  0.9090925 -3.3634611 -1.4196012 -3.5824183
[8] -1.3556774

return_mean1          44.416250
return_mean2          59.544906
return_sd1            88.414632
return_sd2            95.135195
return_skew1         160.667674
return_skew2         217.040922
return_kurtosis1    1110.728

return_mean1          46.688827
return_mean2          60.244925
return_sd1            89.951415
return_sd2            97.046363
return_skew1         159.845918
return_skew2         218.427391
return_kurtosis1    1129.383119
return_kurtosis2    1719.944171
dtype: float64
3521.5321277110347
---
[1]  4.1337739  0.9921593  4.4425780  0.9090925 -3.3634611 -1.6629831 -3.5824183
[8] -1.3556774

return_mean1          45.251516
return_mean2          60.464430
return_sd1           131.709988
return_sd2            96.852344
return_skew1         162.007204
return_skew2         215.196095
return_kurtosis1    1125.467555
return_kurtosis2    1715.138399
dtype: float64
3552.0875313307915
---
[1]  4.1337739  0.9921593  4.4425780  0.9090925 -3.3634611 -1.5104191 -3.5824183
[8] -1.3556774

return_mean1          45.250162
return_mean2          61.509928
return_sd1           100.616100
return_sd2            95.476815
return_skew1         159.783205
return_skew2         216.362941
return_kurtosis1    1125.8

return_mean1          45.882584
return_mean2          60.017281
return_sd1            94.313973
return_sd2            97.329232
return_skew1         159.779671
return_skew2         221.384941
return_kurtosis1    1124.117937
return_kurtosis2    1717.698381
dtype: float64
3520.5239999575415
---
[1]  4.1337739  0.9921593  4.4425780  0.9090925 -3.3634611 -1.4675419 -3.4365203
[8] -1.3556774

return_mean1          46.468289
return_mean2          61.436981
return_sd1            91.678600
return_sd2            93.323526
return_skew1         163.328509
return_skew2         213.541383
return_kurtosis1    1128.082795
return_kurtosis2    1719.575142
dtype: float64
3517.435224536366
---
[1]  4.1337739  0.9921593  4.4425780  0.9090925 -3.3634611 -1.4675419 -3.6409819
[8] -1.3556774

return_mean1          46.808074
return_mean2          58.836994
return_sd1            93.824557
return_sd2            93.856998
return_skew1         163.734194
return_skew2         220.318648
return_kurtosis1    1125.97

return_mean1          45.787476
return_mean2          59.777255
return_sd1            93.883823
return_sd2            95.298021
return_skew1         159.899688
return_skew2         218.007241
return_kurtosis1    1126.872524
return_kurtosis2    1718.461038
dtype: float64
3517.987065493524
---
[1]  4.1337739  0.9921593  4.4425780  0.9090925 -3.3634611 -1.4675419 -3.6050819
[8] -1.9737113

return_mean1          43.872200
return_mean2          58.567220
return_sd1            93.968235
return_sd2           221.254029
return_skew1         160.070898
return_skew2         215.477275
return_kurtosis1    1125.619993
return_kurtosis2    1717.393896
dtype: float64
3636.223746100172
---
[1]  4.1337739  0.9921593  4.4425780  0.9090925 -3.3634611 -1.4675419 -3.6050819
[8] -0.9737114

return_mean1          45.821637
return_mean2          63.191055
return_sd1            92.583100
return_sd2           185.239793
return_skew1         157.743513
return_skew2         213.733639
return_kurtosis1    1120.576

In [2]:
n_sim_ou_log_prices = n_ou_simulation(
    random_seed=9868, num_sim=248,
    
    mu11=4.1337739, mu12=0.9921593, 
    mu21=4.4425780, mu22=0.9090925,
    
    sigma11=-3.3634611, sigma12=-1.4675419, 
    sigma21=-3.6050819, sigma22=-1.4157451,
    
    xinit_vec=xinit_vec, T0=0, T=1, length=250)

n_sim_ou_pair_prices = log_price_to_price(n_log_price=n_sim_ou_log_prices)
n_sim_ou_pair_returns = price_to_return(n_price=n_sim_ou_pair_prices)

print(n_sim_ou_log_prices)
print(n_sim_ou_pair_prices)
print(n_sim_ou_pair_returns)

n_sim_ou_pair_prices.to_csv("/Users/changmao/Desktop/OneDrive - Imperial College London/InferStat - MSc Summer Project/GitHub/Summer-Research-Project/Calibration/Cluster jobs/ou8/n_sim_ou_pair_prices.csv")
n_sim_ou_pair_returns.to_csv("/Users/changmao/Desktop/OneDrive - Imperial College London/InferStat - MSc Summer Project/GitHub/Summer-Research-Project/Calibration/Cluster jobs/ou8/n_sim_ou_pair_returns.csv")




          0         1         2         3         4         5         6    \
0    3.719609  4.123019  4.406780  4.090375  7.080876  5.019460  6.978409   
1    3.724690  4.128633  4.388379  4.074135  7.056739  5.004697  6.963394   
2    3.742179  4.147615  4.377428  4.066538  7.038188  4.997307  6.956862   
3    3.725033  4.130879  4.373845  4.066577  7.002824  4.972357  6.961694   
4    3.700418  4.106173  4.380551  4.077845  7.004385  4.985304  6.945218   
..        ...       ...       ...       ...       ...       ...       ...   
246  4.173578  4.747267  4.092743  4.387367  5.552752  5.255105  5.214363   
247  4.187496  4.762206  4.083106  4.378753  5.565121  5.271562  5.237141   
248  4.212344  4.787852  4.070007  4.366794  5.542351  5.252417  5.213340   
249  4.218483  4.795584  4.050330  4.347058  5.550015  5.263854  5.210279   
250  4.210483  4.787539  4.068073  4.367290  5.561985  5.281408  5.206181   

          7         8         9    ...       486       487       488  \
0  

In [1]:
import pandas as pd
import numpy as np
import random
from numpy.random import RandomState
from scipy import stats
from scipy.optimize import minimize
import scipy
import os
os.environ['R_HOME'] = '/Library/Frameworks/R.framework/Versions/4.1/Resources/'
import rpy2
from rpy2.robjects.packages import importr
from rpy2.robjects.packages import SignatureTranslatedAnonymousPackage
import datetime
from rpy2.robjects.vectors import FloatVector
from statsmodels.tsa.stattools import adfuller
from sklearn.linear_model import LinearRegression

np.random.seed(881)

# Define the model that generates pair simulations.
yuima = importr("yuima")
n_ou_sim_string = """
n_sim_ou = function(random_seed, num_sim,
                    mu11, mu12, mu21, mu22, sigma11, sigma12, sigma21, sigma22,
                    xinit_vec, T0, T, length){

  set.seed(random_seed)

  drift = c("mu11-mu12*X1", "mu21-mu22*X2")
  diffusion = matrix(c("exp(sigma11)", "exp(sigma12)", "exp(sigma21)", "exp(sigma22)"), 2, 2, byrow=TRUE)
  ou_model = setModel(drift=drift, diffusion=diffusion, 
                        time.variable = "t",
                        state.var=c("X1","X2"), solve.variable=c("X1","X2"))

  newsamp = setSampling(Initial=T0, Terminal=T, n=length)

  n_sim_ou_data = data.frame(matrix(nrow=length+1, ncol=2*num_sim))
  for (i in 1:num_sim){
    ou_sim = simulate(ou_model, 
                      true.par=list(
                        mu11=mu11, mu12=mu12, mu21=mu21, mu22=mu22, 
                        sigma11=sigma11, sigma12=sigma12, sigma21=sigma21, sigma22=sigma22), 
                      xinit=xinit_vec[i], sampling=newsamp)
    original_data = ou_sim@data@original.data
    one_sim_ou = data.frame(original_data[,1], original_data[,2])
    colnames(one_sim_ou) = c('series1', 'series2')
    n_sim_ou_data[, (2*i-1):(2*i)] = one_sim_ou
  }
  return(n_sim_ou_data)
}
"""

n_ou_sim = SignatureTranslatedAnonymousPackage(n_ou_sim_string, "n_ou_sim")
def n_ou_simulation(random_seed, num_sim,
                    mu11, mu12, mu21, mu22, sigma11, sigma12, sigma21, sigma22,
                    xinit_vec, T0, T, length):
    """num_sim simulations of bivariate Ornstein-Uhlenbeck process,
    length = length of one series
    """
    n_ou_sim_data = pd.DataFrame(
        n_ou_sim.n_sim_ou(random_seed=random_seed, num_sim=num_sim,
                              mu11=mu11, mu12=mu12, mu21=mu21, mu22=mu22,
                              sigma11=sigma11, sigma12=sigma12, sigma21=sigma21, sigma22=sigma22,
                              xinit_vec=xinit_vec, T0=T0, T=T, length=length)).transpose()
    return n_ou_sim_data

def price_to_log_price(n_price):
    return(np.log(n_price))

def log_price_to_price(n_log_price):
    return(np.exp(n_log_price))

def price_to_return(n_price):
    n_return = pd.DataFrame()
    for i in range(n_price.shape[1]):
        ith_column_price_series = n_price.iloc[:, i]
        n_return = pd.concat([n_return, 100 * (np.log(ith_column_price_series[1:].values) - np.log(ith_column_price_series[:-1]))], axis=1)
    return n_return

def log_price_to_return(n_log_price):
    n_real_return = pd.DataFrame()
    for i in range(n_log_price.shape[1]):
        ith_column_price_series = n_log_price.iloc[:, i]
        n_real_return = pd.concat([n_real_return, 100 * (ith_column_price_series[1:].values - ith_column_price_series[:-1])], axis=1)
    return n_real_return


def cross_corr_coef(lag_time_series, lead_time_series, lag):
    # calculate the cross correlation between two time series
    # if the result is not zero, then 'lead_time_series' leads 'lag_time_series'
    lag_time_series = lag_time_series.iloc[lag:]
    corr_coef = np.corrcoef(lag_time_series, lead_time_series.iloc[0:((lead_time_series.size)-lag)])[0][1]
    return corr_coef

def cal_stats(n_return, n_price):
    
    return_series1 = n_return.iloc[:, ::2]
    return_series2 = n_return.iloc[:, 1::2]
    price_series1 = n_price.iloc[:, ::2]
    price_series2 = n_price.iloc[:, 1::2]

    
    mean1 = return_series1.mean(axis=0).values
    sd1 = return_series1.std(axis=0).values
    skew1 = return_series1.skew(axis=0).values
    kurtosis1 = return_series1.kurtosis(axis=0).values
    autocorr1 = return_series1.apply(lambda x: x.autocorr(lag=1))    
    
    
    mean2 = return_series2.mean(axis=0).values
    sd2 = return_series2.std(axis=0).values
    skew2 = return_series1.skew(axis=0).values
    kurtosis2 = return_series2.kurtosis(axis=0).values
    autocorr2 = return_series2.apply(lambda x: x.autocorr(lag=1)) 
    
    
    # Cross-correlation between return series
    corr_ts1_lag_0 = []
    corr_ts1_lag_1 = [] # if not zero, return series 2 leads return series 1
    corr_ts1_lag_2 = []
    corr_ts1_lag_3 = []
    corr_ts2_lag_1 = [] # if not zero, return series 1 lead return series 2
    corr_ts2_lag_2 = []
    corr_ts2_lag_3 = []
    for i in range(int(real_price.shape[1]/2)):
        corr_ts1_lag_0.append(cross_corr_coef(return_series1.iloc[:, i], return_series2.iloc[:, i], 0))
        corr_ts1_lag_1.append(cross_corr_coef(return_series1.iloc[:, i], return_series2.iloc[:, i], 1))
        corr_ts1_lag_2.append(cross_corr_coef(return_series1.iloc[:, i], return_series2.iloc[:, i], 2))
        corr_ts1_lag_3.append(cross_corr_coef(return_series1.iloc[:, i], return_series2.iloc[:, i], 3))
        corr_ts2_lag_1.append(cross_corr_coef(return_series2.iloc[:, i], return_series1.iloc[:, i], 1))
        corr_ts2_lag_2.append(cross_corr_coef(return_series2.iloc[:, i], return_series1.iloc[:, i], 2))
        corr_ts2_lag_3.append(cross_corr_coef(return_series2.iloc[:, i], return_series1.iloc[:, i], 3))
    corr_ts1_lag_0 = pd.Series(corr_ts1_lag_0)
    corr_ts1_lag_1 = pd.Series(corr_ts1_lag_1)
    corr_ts1_lag_2 = pd.Series(corr_ts1_lag_2)
    corr_ts1_lag_3 = pd.Series(corr_ts1_lag_3)
    corr_ts2_lag_1 = pd.Series(corr_ts2_lag_1)
    corr_ts2_lag_2 = pd.Series(corr_ts2_lag_2)
    corr_ts2_lag_3 = pd.Series(corr_ts2_lag_3)
    
    
    # adf test on original prices: test if the price difference of the two time series is stationary
    p_values = []
    for i in range(int(real_price.shape[1]/2)):
        reg = LinearRegression().fit(np.array(price_series1.iloc[:, i]).reshape((-1, 1)), np.array(price_series2.iloc[:, i]))
        y_pred = reg.predict(np.array(price_series1.iloc[:, i]).reshape((-1, 1)))
        res = np.array(price_series2.iloc[:, i]) - y_pred
        adf_result = adfuller(res)[1]
        p_values.append(adf_result)
    p_values = pd.Series(p_values)
    
    
    stats_data = pd.DataFrame([mean1, mean2, sd1, sd2, 
                               skew1, skew2, kurtosis1, kurtosis2, 
                               autocorr1, autocorr2, corr_ts1_lag_0, 
                               corr_ts1_lag_1, corr_ts1_lag_2, corr_ts1_lag_3,
                               corr_ts2_lag_1, corr_ts2_lag_2, corr_ts2_lag_3,
                               p_values])
    stats_data = stats_data.transpose()
    stats_data.columns = [
        'return_mean1', 'return_mean2',
        'return_sd1', 'return_sd2',
        'return_skew1', 'return_skew2',
        'return_kurtosis1', 'return_kurtosis2',
        'return_autocorrelation1', 'return_autocorrelation2',
        'return_correlation_ts1_lag_0',
        'return_correlation_ts1_lag_1', 'return_correlation_ts1_lag_2', 'return_correlation_ts1_lag_3',
        'return_correlation_ts2_lag_1', 'return_correlation_ts2_lag_2', 'return_correlation_ts2_lag_3',
        'price_adf_p_values']
    return stats_data

def loss_function(params):
    """n_real_stats is a global amount calculated outside the function"""
    params = FloatVector(params)
    print(params)
    moment_loss = pd.DataFrame().reindex_like(real_stats)


    n_sim_log_price = n_ou_simulation(
        random_seed=int(np.random.randint(low=0, high=980608, size=(1,))), num_sim=num_sim,
        mu11=params[0], mu12=params[1], mu21=params[2], mu22=params[3],
        sigma11=params[4], sigma12=params[5], sigma21=params[6], sigma22=params[7],
        xinit_vec=xinit_vec, T0=T0, T=T, length=length)
    n_sim_price = log_price_to_price(n_sim_log_price)
    n_sim_return = price_to_return(n_sim_price)
    n_sim_stats = cal_stats(n_sim_return, n_sim_price)


    for i in range(n_real_stats.shape[0]):
        for j in range(n_real_stats.shape[1]):
            moment_loss.iloc[i, j] = np.sqrt((n_real_stats.iloc[i, j] - n_sim_stats.iloc[i, j])**2)
    sum_all = np.sum(moment_loss)
    print(sum_all)
    print(np.sum(sum_all))
    print('---')

    return np.sum(sum_all)



real_price = pd.read_csv("sp500_20180101_20181231_pair_prices.csv", index_col=[0])
real_log_price = price_to_log_price(n_price=real_price)
real_return = pd.read_csv("sp500_20180101_20181231_pair_returns.csv", index_col=[0])
real_stats = cal_stats(n_return=real_return, n_price=real_price)


xinit_vec = []
for i in range(int(real_log_price.shape[1]/2)):
    init_pair_log_price = [real_log_price.iloc[0, 2*i], real_log_price.iloc[0, 2*i+1]]
    init_pair_log_price = FloatVector(init_pair_log_price)
    xinit_vec.append(init_pair_log_price)
num_sim, T0, T, length = real_stats.shape[0], 0, 1, real_price.shape[0]

n_real_stats = real_stats



initial0 = [1, 1, 1, 1, -1, -1, -1, -1]
begin_time = datetime.datetime.now()
res = minimize(loss_function, initial0, method='Powell',
               tol=1e-6, options={'disp': True},
               bounds=[(0., None), (0., None), (0., None), (0., None),
                       (None, None), (None, None), (None, None), (None, None)])
print(res.x)

time = datetime.datetime.now() - begin_time
print(time)

params = (res.x)
loss = loss_function((params))
print(loss)

[1]  1  1  1  1 -1 -1 -1 -1

return_mean1                     201.809004
return_mean2                     190.217593
return_sd1                       391.363406
return_sd2                       380.505649
return_skew1                     161.925681
return_skew2                     161.925681
return_kurtosis1                1117.956703
return_kurtosis2                1709.255110
return_autocorrelation1           17.960991
return_autocorrelation2           18.254671
return_correlation_ts1_lag_0     157.584948
return_correlation_ts1_lag_1      16.998531
return_correlation_ts1_lag_2      18.999552
return_correlation_ts1_lag_3      18.444276
return_correlation_ts2_lag_1      18.773415
return_correlation_ts2_lag_2      19.772974
return_correlation_ts2_lag_3      17.891755
price_adf_p_values                59.114730
dtype: float64
4678.754670046529
---
[1]  1.115092  1.000000  1.000000  1.000000 -1.000000 -1.000000 -1.000000
[8] -1.000000

return_mean1                     199.897227
return_me

return_mean1                      60.494163
return_mean2                     191.198529
return_sd1                       392.924069
return_sd2                       383.487096
return_skew1                     155.804377
return_skew2                     155.804377
return_kurtosis1                1122.082663
return_kurtosis2                1714.908632
return_autocorrelation1           17.410997
return_autocorrelation2           16.819664
return_correlation_ts1_lag_0     157.181765
return_correlation_ts1_lag_1      15.788649
return_correlation_ts1_lag_2      17.641951
return_correlation_ts1_lag_3      18.107805
return_correlation_ts2_lag_1      17.094856
return_correlation_ts2_lag_2      18.092046
return_correlation_ts2_lag_3      18.567693
price_adf_p_values                42.914580
dtype: float64
4516.323911661465
---
[1]  3.647563  1.000000  1.000000  1.000000 -1.000000 -1.000000 -1.000000
[8] -1.000000

return_mean1                      61.029964
return_mean2                     193.8

return_mean1                      57.556018
return_mean2                     191.038016
return_sd1                       391.533784
return_sd2                       381.851536
return_skew1                     158.384296
return_skew2                     158.384296
return_kurtosis1                1119.233919
return_kurtosis2                1710.303917
return_autocorrelation1           18.161032
return_autocorrelation2           17.113156
return_correlation_ts1_lag_0     157.180065
return_correlation_ts1_lag_1      17.626049
return_correlation_ts1_lag_2      17.883984
return_correlation_ts1_lag_3      19.187241
return_correlation_ts2_lag_1      18.423373
return_correlation_ts2_lag_2      17.670960
return_correlation_ts2_lag_3      19.521427
price_adf_p_values                42.469074
dtype: float64
4513.522143548726
---
[1]  3.699781  1.000000  1.000000  1.000000 -1.000000 -1.000000 -1.000000
[8] -1.000000

return_mean1                      57.039194
return_mean2                     191.8

return_mean1                      57.513001
return_mean2                     193.300525
return_sd1                       394.193109
return_sd2                       383.612625
return_skew1                     160.168588
return_skew2                     160.168588
return_kurtosis1                1127.846609
return_kurtosis2                1719.210479
return_autocorrelation1           18.871266
return_autocorrelation2           18.104131
return_correlation_ts1_lag_0     157.181191
return_correlation_ts1_lag_1      18.308929
return_correlation_ts1_lag_2      18.241270
return_correlation_ts1_lag_3      18.421579
return_correlation_ts2_lag_1      18.853074
return_correlation_ts2_lag_2      17.973954
return_correlation_ts2_lag_3      19.463407
price_adf_p_values                47.942499
dtype: float64
4549.37482380804
---
[1]  3.699771  1.000000  1.000000  1.000000 -1.000000 -1.000000 -1.000000
[8] -1.000000

return_mean1                      57.548960
return_mean2                     187.66

return_mean1                      48.900966
return_mean2                     190.932319
return_sd1                       388.643383
return_sd2                       379.623022
return_skew1                     159.501422
return_skew2                     159.501422
return_kurtosis1                1119.989794
return_kurtosis2                1712.823302
return_autocorrelation1           17.461435
return_autocorrelation2           17.129764
return_correlation_ts1_lag_0     157.045971
return_correlation_ts1_lag_1      16.905584
return_correlation_ts1_lag_2      17.052422
return_correlation_ts1_lag_3      18.980684
return_correlation_ts2_lag_1      17.803010
return_correlation_ts2_lag_2      17.490006
return_correlation_ts2_lag_3      18.810711
price_adf_p_values                42.389522
dtype: float64
4500.984739773259
---
[1]  3.6997809  0.9135521  1.0000000  1.0000000 -1.0000000 -1.0000000 -1.0000000
[8] -1.0000000

return_mean1                      48.105789
return_mean2                  

...

return_mean1                      43.542446
return_mean2                      51.502494
return_sd1                        85.952629
return_sd2                        95.717669
return_skew1                     159.575224
return_skew2                     159.575224
return_kurtosis1                1113.710655
return_kurtosis2                1706.717012
return_autocorrelation1           17.691731
return_autocorrelation2           17.622601
return_correlation_ts1_lag_0     121.205942
return_correlation_ts1_lag_1      17.171469
return_correlation_ts1_lag_2      18.393751
return_correlation_ts1_lag_3      17.943044
return_correlation_ts2_lag_1      19.220507
return_correlation_ts2_lag_2      18.586378
return_correlation_ts2_lag_3      19.038884
price_adf_p_values                82.698973
dtype: float64
3765.8666342096317
---
[1]  4.1008273  0.9141363  5.4960348  1.2456121 -3.8558552 -1.3274259 -1.8739748
[8] -1.5689857

return_mean1                      43.975910
return_mean2                 

return_mean1                      45.637496
return_mean2                      51.423792
return_sd1                        84.745495
return_sd2                        94.755554
return_skew1                     157.542143
return_skew2                     157.542143
return_kurtosis1                1124.998861
return_kurtosis2                1716.531270
return_autocorrelation1           19.163179
return_autocorrelation2           18.217596
return_correlation_ts1_lag_0     121.506043
return_correlation_ts1_lag_1      17.803140
return_correlation_ts1_lag_2      16.744163
return_correlation_ts1_lag_3      19.399035
return_correlation_ts2_lag_1      18.613492
return_correlation_ts2_lag_2      18.267000
return_correlation_ts2_lag_3      18.864460
price_adf_p_values                77.501222
dtype: float64
3779.256085028525
---
[1]  4.1008273  0.9141363  5.4960348  1.2456121 -3.8558552 -1.3274259 -1.8950882
[8] -1.5689857

return_mean1                      43.283752
return_mean2                  

return_mean1                      46.924075
return_mean2                      51.859518
return_sd1                        84.279336
return_sd2                        94.459102
return_skew1                     159.645867
return_skew2                     159.645867
return_kurtosis1                1123.224658
return_kurtosis2                1716.186290
return_autocorrelation1           18.526335
return_autocorrelation2           17.813274
return_correlation_ts1_lag_0     121.002215
return_correlation_ts1_lag_1      16.775250
return_correlation_ts1_lag_2      17.979458
return_correlation_ts1_lag_3      19.054002
return_correlation_ts2_lag_1      19.499798
return_correlation_ts2_lag_2      18.827004
return_correlation_ts2_lag_3      18.549728
price_adf_p_values                78.696942
dtype: float64
3782.9487154076464
---
[1]  4.1008273  0.9141363  5.4960348  1.2456121 -3.8558552 -1.3274259 -1.8950882
[8] -0.5689857

return_mean1                      46.468204
return_mean2                 

return_mean1                      46.878664
return_mean2                      50.724302
return_sd1                        84.363936
return_sd2                       121.210181
return_skew1                     159.612225
return_skew2                     159.612225
return_kurtosis1                1127.024713
return_kurtosis2                1717.944575
return_autocorrelation1           17.946286
return_autocorrelation2           16.566472
return_correlation_ts1_lag_0      98.298900
return_correlation_ts1_lag_1      17.055106
return_correlation_ts1_lag_2      18.312204
return_correlation_ts1_lag_3      19.254141
return_correlation_ts2_lag_1      18.073111
return_correlation_ts2_lag_2      17.459762
return_correlation_ts2_lag_3      19.190852
price_adf_p_values                78.456906
dtype: float64
3787.984560947736
---
[1]  4.1008273  0.9141363  5.4960348  1.2456121 -3.8558552 -1.3274259 -1.8950882
[8] -1.8688283

return_mean1                      44.319838
return_mean2                  

return_mean1                      45.733302
return_mean2                      52.464477
return_sd1                        86.535350
return_sd2                       114.165631
return_skew1                     157.599346
return_skew2                     157.599346
return_kurtosis1                1116.364138
return_kurtosis2                1717.245445
return_autocorrelation1           17.931452
return_autocorrelation2           18.412043
return_correlation_ts1_lag_0     103.452203
return_correlation_ts1_lag_1      18.453895
return_correlation_ts1_lag_2      17.299985
return_correlation_ts1_lag_3      19.580391
return_correlation_ts2_lag_1      18.645195
return_correlation_ts2_lag_2      18.431734
return_correlation_ts2_lag_3      18.700007
price_adf_p_values                79.709627
dtype: float64
3778.3235669133696
---
[1]  4.1008273  0.9141363  5.4960348  1.2456121 -3.8558552 -1.3274259 -1.8950882
[8] -1.8392423

return_mean1                      46.457187
return_mean2                 

In [3]:
real_price = pd.read_csv("/Users/changmao/Desktop/OneDrive - Imperial College London/InferStat - MSc Summer Project/GitHub/Summer-Research-Project/Data/sp500_20190101_20191231/sp500_20190101_20191231_pair_prices.csv", index_col=[0])
real_log_price = price_to_log_price(n_price=real_price)
real_return = pd.read_csv("/Users/changmao/Desktop/OneDrive - Imperial College London/InferStat - MSc Summer Project/GitHub/Summer-Research-Project/Data/sp500_20190101_20191231/sp500_20190101_20191231_pair_returns.csv", index_col=[0])
real_stats = cal_stats(n_return=real_return, n_price=real_price)


xinit_vec = []
for i in range(int(real_log_price.shape[1]/2)):
    init_pair_log_price = [real_log_price.iloc[0, 2*i], real_log_price.iloc[0, 2*i+1]]
    init_pair_log_price = FloatVector(init_pair_log_price)
    xinit_vec.append(init_pair_log_price)
    
num_sim, T0, T, length = real_stats.shape[0], 0, 1, real_price.shape[0]


n_sim_ou_log_prices = n_ou_simulation(
    random_seed=9868, num_sim=248,
    
    mu11=4.1337739, mu12=0.9921593, 
    mu21=4.4425780, mu22=0.9090925,
    
    sigma11=-3.3634611, sigma12=-1.4675419, 
    sigma21=-3.6050819, sigma22=-1.4157451,
    
    xinit_vec=xinit_vec, T0=0, T=1, length=250)

n_sim_ou_pair_prices = log_price_to_price(n_log_price=n_sim_ou_log_prices)
n_sim_ou_pair_returns = price_to_return(n_price=n_sim_ou_pair_prices)

print(n_sim_ou_log_prices)
print(n_sim_ou_pair_prices)
print(n_sim_ou_pair_returns)

n_sim_ou_pair_prices.to_csv("/Users/changmao/Desktop/OneDrive - Imperial College London/InferStat - MSc Summer Project/GitHub/Summer-Research-Project/Calibration/Cluster jobs/ou8/n_sim_ou_pair_prices.csv")
n_sim_ou_pair_returns.to_csv("/Users/changmao/Desktop/OneDrive - Imperial College London/InferStat - MSc Summer Project/GitHub/Summer-Research-Project/Calibration/Cluster jobs/ou8/n_sim_ou_pair_returns.csv")


          0         1         2         3         4         5         6    \
0    3.647595  4.134696  4.584573  4.021957  7.338973  5.094021  6.960993   
1    3.652961  4.140269  4.565466  4.005966  7.313811  5.078987  6.946046   
2    3.670735  4.159208  4.553812  3.998617  7.294240  5.071327  6.939583   
3    3.653872  4.142430  4.549529  3.998903  7.257860  5.046108  6.944484   
4    3.629540  4.117682  4.555538  4.010417  7.258408  5.058787  6.928076   
..        ...       ...       ...       ...       ...       ...       ...   
246  4.146502  4.752033  4.159589  4.359444  5.649791  5.285535  5.207815   
247  4.160527  4.766954  4.149687  4.350932  5.661774  5.301881  5.230619   
248  4.185483  4.792584  4.136324  4.339074  5.638620  5.282627  5.206843   
249  4.191729  4.800299  4.116383  4.319438  5.645903  5.293954  5.203808   
250  4.183834  4.792236  4.133864  4.339771  5.657492  5.311398  5.199736   

          7         8         9    ...       486       487       488  \
0  